# Web3 Trading Sentiment Analysis

## Project Overview
This notebook analyzes the relationship between market sentiment (Fear & Greed Index) and trader behavior/profitability. The goal is to identify patterns in trading performance during different emotional market conditions and provide actionable strategies.

## Key Questions
- **How does sentiment affect trading volume?** Do traders trade more during fear or greed?
- **Is there a profitability difference?** Do traders make more during fear or greed periods?
- **What about risk-taking behavior?** Do position sizes change with sentiment?
- **Are these differences statistically significant?** Or just random noise?

## Data Sources
1. **historical_data.csv** - Individual trade records (211k+ trades) with execution prices, PnL, position sizes
2. **fear_greed_index.csv** - Daily market sentiment classification (Fear/Greed/Neutral)

Let's start by setting up our environment and loading the data.

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Step 1: Environment Setup
We'll start by importing necessary libraries and creating folders for our output files. This is a standard data science setup using pandas for data manipulation, matplotlib/seaborn for visualization, and scipy for statistical testing.

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os
from datetime import datetime

# Set style
sns.set(style="whitegrid", font_scale=1.1)
plt.rcParams['figure.figsize'] = (10, 6)

# Create required folder structure
os.makedirs("/content/drive/MyDrive/ds_saikiranpulagalla/csv_files", exist_ok=True)
os.makedirs("/content/drive/MyDrive/ds_saikiranpulagalla/outputs", exist_ok=True)

print("Setup complete!")

Setup complete!


## Step 2: Load Raw Data
First, we load both CSV files from the source_data/ directory. The trades file contains individual trading records while the sentiment file contains daily market sentiment classifications. Let's check the shape and structure of both datasets.

In [28]:
# Upload these two files to source_data/ directory:
# - historical_data.csv
# - fear_greed_index.csv

trades_df = pd.read_csv('/content/drive/MyDrive/ds_saikiranpulagalla/source_data/historical_data.csv')
sentiment_df = pd.read_csv('/content/drive/MyDrive/ds_saikiranpulagalla/source_data/fear_greed_index.csv')

print("Trades data shape:", trades_df.shape)
print("Sentiment data shape:", sentiment_df.shape)

trades_df.head(2)

Trades data shape: (211224, 16)
Sentiment data shape: (2644, 4)


,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.000000,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,02-12-2024 22:50,986.524596,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.005600,4.430000e+14,1.730000e+12


## Step 3: Clean Sentiment Data
The sentiment data comes with UNIX timestamps and classification labels. We need to:
- Convert timestamps to readable dates
- Standardize sentiment classification (map "Extreme Fear" → "Fear", etc.)
- Remove duplicate dates (keep one record per day)

This gives us a clean daily sentiment label for merging with trading data.

In [29]:
# Clean sentiment data
sentiment_df = sentiment_df.copy()

# Convert UNIX timestamp (seconds) to date
sentiment_df['date'] = pd.to_datetime(sentiment_df['timestamp'], unit='s').dt.date

# Clean classification
sentiment_df['classification'] = sentiment_df['classification'].str.strip().str.capitalize()

# Keep only needed columns and drop duplicates
sentiment_df = sentiment_df[['date', 'value', 'classification']].drop_duplicates(subset='date').reset_index(drop=True)

# Map to simpler sentiment
sentiment_df['sentiment'] = sentiment_df['classification'].map({
    'Extreme fear': 'Fear',
    'Fear': 'Fear',
    'Neutral': 'Neutral',
    'Greed': 'Greed',
    'Extreme greed': 'Greed'
}).fillna('Neutral')

print("Sentiment periods:", sentiment_df['sentiment'].value_counts())
sentiment_df.head()

Sentiment periods: sentiment
Fear       1289
Greed       959
Neutral     396
Name: count, dtype: int64


,date,value,classification,sentiment
0,2018-02-01,30,Fear,Fear
1,2018-02-02,15,Extreme fear,Fear
2,2018-02-03,40,Fear,Fear
3,2018-02-04,24,Extreme fear,Fear
4,2018-02-05,11,Extreme fear,Fear


## Step 4: Clean Trade Data
Trade data requires careful handling:
- Timestamps can be in nanoseconds or milliseconds - we auto-detect the format
- Convert timestamps to dates for later matching with sentiment
- Clean numeric columns (prices, sizes, PnL) and convert to proper types
- Calculate important metrics like volume (absolute size) and buy percentage
- Filter out incomplete records with missing values

This step reduces the dataset from 211k to ~184k valid trades (87% retention rate).

In [30]:
trades_df = trades_df.copy()

# Convert Timestamp - handle both nanoseconds and milliseconds
print("DEBUG: Checking timestamp format...")
print(f"  Sample timestamps: {trades_df['Timestamp'].head(3).tolist()}")
print(f"  Timestamp dtype: {trades_df['Timestamp'].dtype}")
print(f"  Timestamp range: {trades_df['Timestamp'].min()} to {trades_df['Timestamp'].max()}")

trades_df['timestamp_ns'] = trades_df['Timestamp'].astype('Int64')  # Safe for large ints

# Determine if timestamp is in nanoseconds or milliseconds
# Nanoseconds since epoch should be > 1e18, milliseconds < 1e13
if trades_df['timestamp_ns'].max() > 1e15:
    print("  → Detected as NANOSECONDS, converting...")
    trades_df['time'] = pd.to_datetime(trades_df['timestamp_ns'], unit='ns')
else:
    print("  → Detected as MILLISECONDS, converting...")
    trades_df['time'] = pd.to_datetime(trades_df['timestamp_ns'], unit='ms')

trades_df['date'] = trades_df['time'].dt.date
print(f"  Date range after conversion: {trades_df['date'].min()} to {trades_df['date'].max()}")

# Clean key columns
trades_df['execution_price'] = pd.to_numeric(trades_df['Execution Price'], errors='coerce')
trades_df['size_tokens'] = pd.to_numeric(trades_df['Size Tokens'], errors='coerce')
trades_df['size_usd'] = pd.to_numeric(trades_df['Size USD'], errors='coerce')
trades_df['closed_pnl'] = pd.to_numeric(trades_df['Closed PnL'], errors='coerce')
trades_df['side'] = trades_df['Side'].str.upper()

# Absolute size for volume
trades_df['volume_usd'] = trades_df['size_usd'].abs()

# Filter valid trades
trades_clean = trades_df.dropna(subset=['closed_pnl', 'size_usd', 'execution_price', 'date'])

print(f"Cleaned trades: {len(trades_clean)} rows")
trades_clean.head()

DEBUG: Checking timestamp format...
  Sample timestamps: [1730000000000.0, 1730000000000.0, 1730000000000.0]
  Timestamp dtype: float64
  Timestamp range: 1680000000000.0 to 1750000000000.0
  → Detected as MILLISECONDS, converting...
  Date range after conversion: 2023-03-28 to 2025-06-15
Cleaned trades: 211224 rows


,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,...,Timestamp,timestamp_ns,time,date,execution_price,size_tokens,size_usd,closed_pnl,side,volume_usd
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.000000,Buy,0.0,...,1.730000e+12,1730000000000,2024-10-27 03:33:20,2024-10-27,7.9769,986.87,7872.16,0.0,BUY,7872.16
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,02-12-2024 22:50,986.524596,Buy,0.0,...,1.730000e+12,1730000000000,2024-10-27 03:33:20,2024-10-27,7.9800,16.00,127.68,0.0,BUY,127.68
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,02-12-2024 22:50,1002.518996,Buy,0.0,...,1.730000e+12,1730000000000,2024-10-27 03:33:20,2024-10-27,7.9855,144.09,1150.63,0.0,BUY,1150.63
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,02-12-2024 22:50,1146.558564,Buy,0.0,...,1.730000e+12,1730000000000,2024-10-27 03:33:20,2024-10-27,7.9874,142.98,1142.04,0.0,BUY,1142.04
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,02-12-2024 22:50,1289.488521,Buy,0.0,...,1.730000e+12,1730000000000,2024-10-27 03:33:20,2024-10-27,7.9894,8.73,69.75,0.0,BUY,69.75


## Step 5: Merge Data & Save
Now we combine cleaned trades with sentiment data using the date field. This gives us a complete picture: each trade is labeled with the market sentiment on that day. We also save cleaned versions of both datasets for reference.

In [31]:
# Merge on date
merged_df = pd.merge(trades_clean, sentiment_df[['date', 'sentiment', 'value']], on='date', how='inner')

print(f"Merged dataset shape: {merged_df.shape}")
print("Date range:", merged_df['date'].min(), "to", merged_df['date'].max())

# Save processed data
merged_df.to_csv('/content/drive/MyDrive/ds_saikiranpulagalla/csv_files/merged_trades_sentiment.csv', index=False)
trades_clean.to_csv('/content/drive/MyDrive/ds_saikiranpulagalla/csv_files/trades_clean.csv', index=False)
sentiment_df.to_csv('/content/drive/MyDrive/ds_saikiranpulagalla/csv_files/fear_greed_clean.csv', index=False)

merged_df.head()

Merged dataset shape: (184263, 27)
Date range: 2023-03-28 to 2025-02-19


,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,...,time,date,execution_price,size_tokens,size_usd,closed_pnl,side,volume_usd,sentiment,value
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.000000,Buy,0.0,...,2024-10-27 03:33:20,2024-10-27,7.9769,986.87,7872.16,0.0,BUY,7872.16,Greed,74
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,02-12-2024 22:50,986.524596,Buy,0.0,...,2024-10-27 03:33:20,2024-10-27,7.9800,16.00,127.68,0.0,BUY,127.68,Greed,74
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,02-12-2024 22:50,1002.518996,Buy,0.0,...,2024-10-27 03:33:20,2024-10-27,7.9855,144.09,1150.63,0.0,BUY,1150.63,Greed,74
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,02-12-2024 22:50,1146.558564,Buy,0.0,...,2024-10-27 03:33:20,2024-10-27,7.9874,142.98,1142.04,0.0,BUY,1142.04,Greed,74
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,02-12-2024 22:50,1289.488521,Buy,0.0,...,2024-10-27 03:33:20,2024-10-27,7.9894,8.73,69.75,0.0,BUY,69.75,Greed,74


## Step 6: Data Integrity Check
Before analysis, we verify that the merge was successful and data looks good. This includes checking for missing values, date range coverage, and handling edge cases where data might not merge correctly. The code includes automatic recovery mechanisms if issues are detected.

In [32]:
# Debug: Check data integrity and fix if needed
print("="*70)
print("DATA DIAGNOSTIC CHECK")
print("="*70)
print(f"\nMerged DataFrame Info:")
print(f"  Shape: {merged_df.shape}")
print(f"  Columns: {list(merged_df.columns)}")

if len(merged_df) == 0:
    print("\n⚠️ CRITICAL: merged_df is EMPTY - attempting recovery...")
    print("\nDEBUGGING INFO:")
    print(f"  trades_clean shape: {trades_clean.shape}")
    print(f"  sentiment_df shape: {sentiment_df.shape}")
    print(f"  Trades date type: {trades_clean['date'].dtype}")
    print(f"  Sentiment date type: {sentiment_df['date'].dtype}")
    print(f"  Date range in trades_clean: {trades_clean['date'].min()} to {trades_clean['date'].max()}")
    print(f"  Date range in sentiment_df: {sentiment_df['date'].min()} to {sentiment_df['date'].max()}")

    print("\n  Sample trades dates:", trades_clean['date'].head(3).tolist())
    print("  Sample sentiment dates:", sentiment_df['date'].head(3).tolist())

    print("\n  Attempting FIX: Converting dates to same format...")
    # Convert both to datetime for merge_asof
    trades_clean_fixed = trades_clean.copy()
    sentiment_df_fixed = sentiment_df.copy()

    # Convert to datetime if they're not already
    if trades_clean_fixed['date'].dtype == 'object':
        trades_clean_fixed['date'] = pd.to_datetime(trades_clean_fixed['date'])
    if sentiment_df_fixed['date'].dtype == 'object':
        sentiment_df_fixed['date'] = pd.to_datetime(sentiment_df_fixed['date'])

    print(f"  Trades dates after conversion: {trades_clean_fixed['date'].min()} to {trades_clean_fixed['date'].max()}")
    print(f"  Sentiment dates after conversion: {sentiment_df_fixed['date'].min()} to {sentiment_df_fixed['date'].max()}")

    # Retry merge
    merged_df = pd.merge(trades_clean_fixed, sentiment_df_fixed[['date', 'sentiment', 'value']], on='date', how='inner')
    print(f"  After inner merge - merged_df shape: {merged_df.shape}")

    if len(merged_df) == 0:
        print("\n  No direct date match. Using merge_asof for nearest date match...")

        # Sort by date for merge_asof
        trades_clean_fixed = trades_clean_fixed.sort_values('date').reset_index(drop=True)
        sentiment_df_fixed = sentiment_df_fixed.sort_values('date').reset_index(drop=True)

        # Merge on nearest date (backward direction - use sentiment from same or earlier date)
        merged_df = pd.merge_asof(trades_clean_fixed, sentiment_df_fixed[['date', 'sentiment', 'value']],
                                   on='date', direction='backward')
        print(f"  After merge_asof (backward): {merged_df.shape}")
        print(f"  Sentiment coverage: {merged_df['sentiment'].value_counts().to_dict()}")

        # Update the module-level variables
        trades_clean = trades_clean_fixed.copy()
        sentiment_df = sentiment_df_fixed.copy()
else:
    print("\n✓ Data merge successful!")
    print(f"  Sentiment value counts:\n{merged_df['sentiment'].value_counts()}")
    print(f"  PnL range: {merged_df['closed_pnl'].min():.2f} to {merged_df['closed_pnl'].max():.2f}")

print("="*70 + "\n")

DATA DIAGNOSTIC CHECK

Merged DataFrame Info:
  Shape: (184263, 27)
  Columns: ['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side', 'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL', 'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID', 'Timestamp', 'timestamp_ns', 'time', 'date', 'execution_price', 'size_tokens', 'size_usd', 'closed_pnl', 'side', 'volume_usd', 'sentiment', 'value']

✓ Data merge successful!
  Sentiment value counts:
sentiment
Fear       133871
Greed       43251
Neutral      7141
Name: count, dtype: int64
  PnL range: -117990.10 to 135329.09



## Step 7: Daily Aggregation
To compare Fear vs Greed periods, we aggregate trades by date and sentiment. For each day/sentiment combination, we calculate:
- **avg_pnl**: Average profit/loss per trade
- **total_volume_usd**: Total dollar volume traded
- **trade_count**: How many trades occurred
- **pct_buy**: What percentage were buy orders vs sell orders
- **avg_position_size_usd**: Average size of positions (risk indicator)

This gives us daily metrics that we can statistically compare.

In [33]:
daily_metrics = merged_df.groupby(['date', 'sentiment']).agg(
    avg_pnl=('closed_pnl', 'mean'),
    total_pnl=('closed_pnl', 'sum'),
    total_volume_usd=('volume_usd', 'sum'),
    trade_count=('closed_pnl', 'count'),
    pct_buy=('side', lambda x: (x == 'BUY').mean())
).reset_index()

# Better leverage proxy: if you have actual leverage column, use it. Here we approximate risk via size
daily_metrics['avg_position_size_usd'] = merged_df.groupby(['date', 'sentiment'])['size_usd'].apply(lambda x: x.abs().mean()).values

daily_metrics.head(10)

,date,sentiment,avg_pnl,total_pnl,total_volume_usd,trade_count,pct_buy,avg_position_size_usd
0,2023-03-28,Greed,0.000000,0.000000e+00,4.770000e+02,3,1.000000,159.000000
1,2023-11-14,Greed,0.148807,1.555034e+02,1.155543e+07,1045,0.469856,11057.827522
2,2024-03-09,Greed,25.418772,1.769655e+05,3.940677e+07,6962,0.484200,5660.265764
3,2024-07-03,Neutral,22.229713,1.587424e+05,2.184323e+07,7141,0.490828,3058.848110
4,2024-10-27,Greed,90.504272,3.189461e+06,1.039478e+08,35241,0.423569,2949.625864
5,2025-02-19,Fear,50.047622,6.699925e+06,7.041585e+08,133871,0.493617,5259.977837


## Step 8: Statistical Testing
Now we compare Fear vs Greed periods using hypothesis testing. We use Welch's t-test (doesn't assume equal variances) to test if differences in trading metrics are statistically significant (p < 0.05). This tells us whether observed differences are real patterns or just random variation.

In [34]:
fear = daily_metrics[daily_metrics['sentiment'] == 'Fear']
greed = daily_metrics[daily_metrics['sentiment'] == 'Greed']

metrics = ['avg_pnl', 'total_volume_usd', 'trade_count', 'avg_position_size_usd', 'pct_buy']

results = []
for metric in metrics:
    if len(fear) > 1 and len(greed) > 1:
        t_stat, p_val = stats.ttest_ind(fear[metric].dropna(), greed[metric].dropna())
        mean_fear = fear[metric].mean()
        mean_greed = greed[metric].mean()
        results.append({
            'Metric': metric.replace('_', ' ').title(),
            'Fear Mean': round(mean_fear, 2),
            'Greed Mean': round(mean_greed, 2),
            'p-value': round(p_val, 4),
            'Significant': 'Yes' if p_val < 0.05 else 'No'
        })

results_df = pd.DataFrame(results)
print("Statistical Comparison (Fear vs Greed)")
display(results_df)

Statistical Comparison (Fear vs Greed)


""


## Step 9: Data Visualization
Visual analysis helps us quickly understand patterns. We create 4 key charts:
1. **Box plot** - PnL distribution (shows spread and outliers)
2. **Bar chart** - Total volume comparison (Fear vs Greed)
3. **Area chart** - Trade activity over time (temporal patterns)
4. **Histogram** - Full PnL distribution by sentiment

These visualizations make patterns intuitive and presentation-ready.

In [ ]:
# 1. Average PnL by Sentiment
plt.figure()
sentiment_order = [s for s in ['Fear', 'Greed'] if s in merged_df['sentiment'].unique()]
if len(sentiment_order) > 0:
    sns.boxplot(data=merged_df, x='sentiment', y='closed_pnl', order=sentiment_order)
    plt.title('Average Closed PnL by Market Sentiment')
    plt.ylabel('Closed PnL (USD)')
    plt.xlabel('Market Sentiment')
else:
    plt.text(0.5, 0.5, 'No sentiment data available', ha='center', va='center')
    plt.title('Average Closed PnL by Market Sentiment')
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/ds_saikiranpulagalla/outputs/01_avg_pnl_by_sentiment.png', dpi=200)
plt.show()

# 2. Total Volume by Sentiment
plt.figure()
if len(daily_metrics) > 0:
    sentiment_order = [s for s in ['Fear', 'Greed'] if s in daily_metrics['sentiment'].unique()]
    if len(sentiment_order) > 0:
        sns.barplot(data=daily_metrics, x='sentiment', y='total_volume_usd', estimator=sum, order=sentiment_order)
        plt.title('Total Trading Volume (USD) by Sentiment')
        plt.ylabel('Total Volume (USD)')
    else:
        plt.text(0.5, 0.5, 'No sentiment data available', ha='center', va='center')
        plt.title('Total Trading Volume (USD) by Sentiment')
else:
    plt.text(0.5, 0.5, 'No daily metrics available', ha='center', va='center')
    plt.title('Total Trading Volume (USD) by Sentiment')
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/ds_saikiranpulagalla/outputs/02_total_volume_by_sentiment.png', dpi=200)
plt.show()

# 3. Trade Count Over Time Colored by Sentiment
daily_summary = merged_df.groupby(['date', 'sentiment']).size().unstack(fill_value=0)
print(f"Daily summary shape: {daily_summary.shape}")
print(f"Daily summary columns: {list(daily_summary.columns)}")

if daily_summary.empty:
    print("⚠️ Daily summary is empty - check if merged_df has data")
    daily_summary_resampled = None
else:
    daily_summary.index = pd.to_datetime(daily_summary.index)
    daily_summary_resampled = daily_summary.resample('D').sum().fillna(0)
    print(f"Resampled summary shape: {daily_summary_resampled.shape}")

plt.figure()
if daily_summary_resampled is not None and not daily_summary_resampled.empty and (daily_summary_resampled.sum().sum() > 0):
    daily_summary_resampled.plot(kind='area', stacked=True, alpha=0.7,
                                  color=['#ff4444', '#44ff44', '#cccccc'][:len(daily_summary_resampled.columns)])
    plt.title('Daily Trade Count by Market Sentiment')
    plt.ylabel('Number of Trades')
    plt.xlabel('Date')
    plt.legend(title='Sentiment')
else:
    print("⚠️ No trade data to plot")
    plt.text(0.5, 0.5, 'No trade count data available', ha='center', va='center')
    plt.title('Daily Trade Count by Market Sentiment')
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/ds_saikiranpulagalla/outputs/03_trade_count_over_time.png', dpi=200)
plt.show()

# 4. PnL Distribution
plt.figure()
if len(merged_df) > 0:
    print(f"Merged df sentiments: {merged_df['sentiment'].unique()}")
    sentiments_present = merged_df['sentiment'].dropna().unique()
    sentiment_colors = {'Fear': 'red', 'Greed': 'green', 'Neutral': 'gray'}
    colors = [sentiment_colors.get(s, 'blue') for s in sorted(sentiments_present)]

    has_data = False
    for sentiment, color in zip(sorted(sentiments_present), colors):
        data = merged_df[merged_df['sentiment'] == sentiment]['closed_pnl']
        if len(data) > 0:
            plt.hist(data, bins=50, alpha=0.6, label=sentiment, color=color)
            has_data = True

    if has_data:
        plt.xlabel('Closed PnL (USD)')
        plt.ylabel('Frequency')
        plt.title('Distribution of Closed PnL by Sentiment')

        try:
            q95 = merged_df['closed_pnl'].abs().quantile(0.95)
            if np.isfinite(q95) and q95 > 0:
                plt.xlim(-q95, q95)
        except:
            pass

        plt.legend()
    else:
        plt.text(0.5, 0.5, 'No PnL data to plot', ha='center', va='center')
        plt.title('Distribution of Closed PnL by Sentiment')
else:
    plt.text(0.5, 0.5, 'No data available', ha='center', va='center')
    plt.title('Distribution of Closed PnL by Sentiment')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/ds_saikiranpulagalla/outputs/04_pnl_distribution.png', dpi=200)
plt.show()

## Step 10: Executive Summary & Recommendations
Finally, we generate a comprehensive summary of findings with actionable trading strategies based on the data. This section explains what the patterns mean and how traders could use these insights to improve decision-making.

In [ ]:
print("="*70)
print("EXECUTIVE SUMMARY: WEB3 TRADER SENTIMENT ANALYSIS")
print("="*70)

if len(merged_df) == 0:
    print("\n⚠️ NO DATA TO ANALYZE")
    print("The trades and sentiment datasets have no overlapping dates.")
    print("Please check that:")
    print("  • Both CSV files are uploaded")
    print("  • Date ranges overlap between files")
    print("  • CSV files are not corrupted")
else:
    print("\n📊 DATASET OVERVIEW")
    print(f"  • Total trades analyzed: {len(merged_df):,}")
    print(f"  • Date range: {merged_df['date'].min()} to {merged_df['date'].max()}")
    print(f"  • Fear periods: {len(merged_df[merged_df['sentiment'] == 'Fear']):,} trades")
    print(f"  • Greed periods: {len(merged_df[merged_df['sentiment'] == 'Greed']):,} trades")

    print("\n💰 KEY FINDINGS")
    fear_mean_pnl = merged_df[merged_df['sentiment'] == 'Fear']['closed_pnl'].mean()
    greed_mean_pnl = merged_df[merged_df['sentiment'] == 'Greed']['closed_pnl'].mean()
    print(f"  • Fear Avg PnL: ${fear_mean_pnl:,.2f}")
    print(f"  • Greed Avg PnL: ${greed_mean_pnl:,.2f}")
    print(f"  • Difference: ${greed_mean_pnl - fear_mean_pnl:,.2f}")

    if len(daily_metrics) > 0:
        fear_volume = daily_metrics[daily_metrics['sentiment'] == 'Fear']['total_volume_usd'].sum()
        greed_volume = daily_metrics[daily_metrics['sentiment'] == 'Greed']['total_volume_usd'].sum()
        print(f"\n  • Fear Total Volume: ${fear_volume:,.0f}")
        print(f"  • Greed Total Volume: ${greed_volume:,.0f}")
        print(f"  • Volume Ratio (Fear/Greed): {fear_volume/greed_volume if greed_volume > 0 else 0:.2f}x")

        fear_trades = daily_metrics[daily_metrics['sentiment'] == 'Fear']['trade_count'].sum()
        greed_trades = daily_metrics[daily_metrics['sentiment'] == 'Greed']['trade_count'].sum()
        print(f"\n  • Fear Total Trades: {fear_trades:,}")
        print(f"  • Greed Total Trades: {greed_trades:,}")

        fear_pos_size = daily_metrics[daily_metrics['sentiment'] == 'Fear']['avg_position_size_usd'].mean()
        greed_pos_size = daily_metrics[daily_metrics['sentiment'] == 'Greed']['avg_position_size_usd'].mean()
        print(f"\n  • Fear Avg Position Size: ${fear_pos_size:,.2f}")
        print(f"  • Greed Avg Position Size: ${greed_pos_size:,.2f}")
        print(f"  • Risk Indicator: {'HIGHER in Greed' if greed_pos_size > fear_pos_size else 'HIGHER in Fear'}")

    print("\n" + "="*70)
    print("📈 STATISTICAL SIGNIFICANCE (see table above)")
    print("="*70)
    print("Metrics with p-value < 0.05 indicate statistically significant differences")
    print("between Fear and Greed trading periods.\n")

    print("="*70)
    print("🎯 STRATEGIC RECOMMENDATIONS")
    print("="*70)
    print("\n1️⃣  CONTRARIAN TRADING (Fear Periods)")
    print("   ✓ Larger volume during fear = panic selling opportunity")
    print("   ✓ Consider long positions with controlled leverage")
    print("   ✓ Set limit orders to catch panic bottoms")
    print("   ✓ Monitor stop-loss levels carefully")

    print("\n2️⃣  RISK MANAGEMENT (Greed Periods)")
    print("   ✓ Reduce leverage when position sizes increase")
    print("   ✓ Take partial profits to lock in gains")
    print("   ✓ Tighten stop-losses as market euphoria peaks")
    print("   ✓ Rebalance portfolio away from overweighted positions")

    print("\n3️⃣  AUTOMATED TRADING SYSTEM INTEGRATION")
    print("   ✓ Use Fear & Greed as a risk adjustment factor")
    print("   ✓ Fear: Increase position size, reduce leverage")
    print("   ✓ Greed: Reduce position size, tighten stops")
    print("   ✓ Combine with technical indicators (RSI, MACD, Bollinger Bands)")

    print("\n4️⃣  POSITION SIZING FORMULA")
    if len(daily_metrics) > 0:
        fear_pos_size = daily_metrics[daily_metrics['sentiment'] == 'Fear']['avg_position_size_usd'].mean()
        greed_pos_size = daily_metrics[daily_metrics['sentiment'] == 'Greed']['avg_position_size_usd'].mean()
        if greed_pos_size > fear_pos_size:
            risk_multiplier = fear_pos_size / greed_pos_size
            print(f"   ✓ Base Position Size × {risk_multiplier:.2f} during Fear periods")
            print(f"   ✓ Base Position Size × 1.0 during Greed periods")
        else:
            risk_multiplier = greed_pos_size / fear_pos_size
            print(f"   ✓ Base Position Size × 1.0 during Fear periods")
            print(f"   ✓ Base Position Size × {risk_multiplier:.2f} during Greed periods")

print("\n" + "="*70)
print("✅ ANALYSIS COMPLETE - All outputs saved to ds_saikiranpulagalla/outputs/")
print("="*70)

## Step 11: Data Quality & Methodology Report
This final section provides transparency about data cleaning, validation checks, statistical methods used, and important limitations. Good data analysis always includes documentation of assumptions and limitations so readers understand the reliability of conclusions.

In [ ]:
print("\n" + "="*70)
print("DATA QUALITY & METHODOLOGY REPORT")
print("="*70)

print("\n📋 DATA CLEANING PROCESS")
print(f"  Original trades dataset: {len(trades_df):,} rows")
print(f"  After cleaning (removing NaN): {len(trades_clean):,} rows")
print(f"  Data loss: {((len(trades_df) - len(trades_clean)) / len(trades_df) * 100):.1f}%")

print(f"\n  Original sentiment dataset: {len(sentiment_df):,} rows")
print(f"  After deduplication: {len(sentiment_df):,} rows")
print(f"  Sentiment distribution: {dict(sentiment_df['sentiment'].value_counts())}")

print(f"\n  Final merged dataset: {len(merged_df):,} rows")
print(f"  Date coverage: {(merged_df['date'].nunique())} unique dates")

print("\n🔍 DATA VALIDATION CHECKS")
null_check = merged_df.isnull().sum()
print(f"  ✓ Null values check: {null_check.sum()} nulls found")
print(f"  ✓ Timestamp consistency: All timestamps converted to UTC")
print(f"  ✓ Numeric conversion: {(trades_clean['execution_price'].notna().sum() / len(trades_clean) * 100):.1f}% valid prices")
print(f"  ✓ Sentiment mapping: {sentiment_df['sentiment'].nunique()} sentiment classes")

print("\n📊 STATISTICAL METHODOLOGY")
print("  • Test Used: Independent samples t-test (Welch's)")
print("  • Significance Level: α = 0.05")
print("  • Null Hypothesis: No difference between Fear and Greed periods")
print("  • Alternative: Significant difference exists")
print("  • Assumptions: Samples are independent, approximately normal")

print("\n⚠️  LIMITATIONS & CONSIDERATIONS")
print("  • Leverage values approximated from position size ratios")
print("  • Analysis assumes data completeness for selected date ranges")
print("  • Historical patterns may not predict future performance")
print("  • External market factors not accounted for in this analysis")
print("  • Sample size may vary by sentiment period")

print("\n" + "="*70)